# ETL Requirements

## Overivew
Objective of the project is to evalute if there is any realtionship beween new cases and weather (temperature and humidity) for two citites New york, Sao Paulo. This notebook provide solution to pull data from the data providers selected (see more information from discovery stage) and transform them into one target file which combines all necessary inputs . Final file is then saved in Resources folder with file name final_combine_data.csv.
Final file is then used for the model phase of the porject.

## Extract requirements

### Data providers:

    1.Extract New york new cases data from NYC Health Git hub repository: https://github.com/nychealth/coronavirus-data
    2.Extract Sao Paulo new cases data from Seade Foundation Statistics Agency of the State of São Paulo: https://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/covid19/				
    3.Load Temperature data using Openweathermap.org history bulk product.www.openweathermap.org
    
### Data Dictionary of Data Sources:

    1.New York meta data https://github.com/nychealth/coronavirus-data/blob/master/trends/Readme.md#cases-by-daycsv
    2.Brazil new cases by cities meta data https://github.com/seade-R/dados-covid-sp#dicion%C3%A1rio-de-vari%C3%A1veis-fontes-prim%C3%A1rias-e-demais-informa%C3%A7%C3%B5es-t%C3%A9cnicas
    3.Weather data from openweathermap.org https://openweathermap.org/history-bulk#examples
 
### Source data file links

    1.New york file link https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/cases-by-day.csv
    2.Brazil city data file link https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv	
    3.Weather data file link: https://history.openweathermap.org/storage/fa037ddb81b7f7f0a0d1a0ebd131858e.csv	
    Note: weather data was one time requested as bulk history up to May 16,2021. If we decide to refresh the model using latest data, then new weather data need to be added either via another history bulk or through daily api



In [20]:
#Import dependancies
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt
%matplotlib inline


# New York Data one time ETL

### Transformation requirements:

#### New york data transformation requirements:
Note: ensure data is ordered by date before running rolling function.

    1.Extract data from source into a data frame
    2.Validate the data types and data for date_of_interest , case_count, CASE_COUNT_7DAY_AVG
    3.Renmae column data_of_interest as "Reported_Date"  
    4.Rename column case_count as "New_Cases"
    5.Rename column CASE_COUNT_7DAY_AVG as "mavg_7day_new_cases"
    6.Add new column City and populate all rows with value "New York"
    7.Add new column is_newyork and populate all rows with value 1
    8.Add new column population and populate with New york city popluating found by googling term "new york city 2020 population" i.e 18804000
    9.Add new column Data_Source and populate it with value https://github.com/nychealth/coronavirus-data/blob/master/trends/data-by-day.csv
    10.Add new column extract_date as todays date time to stamp the date the data is downloaded
    11.Add new calculated column new_cases_per_100K using formula (new_cases/population)*100000
    12.Add new calculated column mavg_7day_per_100k_new_cases using rolling function for pandas on new_cases_per_100K
    13.Change the order of columns "Extract_Date","Reported_Date","City","is_newyork","Population","Data_Source",
                          "New_Cases","mavg_7day_new_cases","new_cases_per_100K","mavg_7day_per_100k_new_cases"
    14.Export the final results as "NYC_Covid_New_Cases_Final.csv" in Resources folder

In [5]:
#1. import directly from GitHub
ny_raw_df = pd.read_csv("https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/cases-by-day.csv",parse_dates=['date_of_interest'])
ny_raw_df.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BK_CASE_COUNT,...,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,INCOMPLETE
0,2020-02-29,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-03,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2020-03-04,5,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,0


In [6]:
#2. check columns data types
ny_raw_df.dtypes

date_of_interest              datetime64[ns]
CASE_COUNT                             int64
PROBABLE_CASE_COUNT                    int64
CASE_COUNT_7DAY_AVG                    int64
ALL_CASE_COUNT_7DAY_AVG                int64
BX_CASE_COUNT                          int64
BX_PROBABLE_CASE_COUNT                 int64
BX_CASE_COUNT_7DAY_AVG                 int64
BX_ALL_CASE_COUNT_7DAY_AVG             int64
BK_CASE_COUNT                          int64
BK_PROBABLE_CASE_COUNT                 int64
BK_CASE_COUNT_7DAY_AVG                 int64
BK_ALL_CASE_COUNT_7DAY_AVG             int64
MN_CASE_COUNT                          int64
MN_PROBABLE_CASE_COUNT                 int64
MN_CASE_COUNT_7DAY_AVG                 int64
MN_ALL_CASE_COUNT_7DAY_AVG             int64
QN_CASE_COUNT                          int64
QN_PROBABLE_CASE_COUNT                 int64
QN_CASE_COUNT_7DAY_AVG                 int64
QN_ALL_CASE_COUNT_7DAY_AVG             int64
SI_CASE_COUNT                          int64
SI_PROBABL

In [7]:
#2.validate
ny_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453 entries, 0 to 452
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_of_interest            453 non-null    datetime64[ns]
 1   CASE_COUNT                  453 non-null    int64         
 2   PROBABLE_CASE_COUNT         453 non-null    int64         
 3   CASE_COUNT_7DAY_AVG         453 non-null    int64         
 4   ALL_CASE_COUNT_7DAY_AVG     453 non-null    int64         
 5   BX_CASE_COUNT               453 non-null    int64         
 6   BX_PROBABLE_CASE_COUNT      453 non-null    int64         
 7   BX_CASE_COUNT_7DAY_AVG      453 non-null    int64         
 8   BX_ALL_CASE_COUNT_7DAY_AVG  453 non-null    int64         
 9   BK_CASE_COUNT               453 non-null    int64         
 10  BK_PROBABLE_CASE_COUNT      453 non-null    int64         
 11  BK_CASE_COUNT_7DAY_AVG      453 non-null    int64         

In [8]:
#2. Statistical summary of raw data
ny_raw_df[["CASE_COUNT","CASE_COUNT_7DAY_AVG"]].describe()

,CASE_COUNT,CASE_COUNT_7DAY_AVG
count,453.000000,453.000000
mean,1726.947020,1725.306843
std,1616.980908,1518.187213
min,0.000000,0.000000
25%,366.000000,349.000000
50%,1049.000000,1054.000000
75%,3005.000000,2915.000000
max,6578.000000,5291.000000


In [9]:
# 2. Select the columns  
ny_transform_df = ny_raw_df[["date_of_interest","CASE_COUNT","CASE_COUNT_7DAY_AVG"]]
ny_transform_df.head()

,date_of_interest,CASE_COUNT,CASE_COUNT_7DAY_AVG
0,2020-02-29,1,0
1,2020-03-01,0,0
2,2020-03-02,0,0
3,2020-03-03,1,0
4,2020-03-04,5,0


In [58]:
#3,4,5 Rename columns
ny_transform_df=ny_transform_df.rename(columns={"date_of_interest":"Reported_Date",
                                               "CASE_COUNT":"New_Cases",
                                               "CASE_COUNT_7DAY_AVG":"mavg_7day_new_cases"})
ny_transform_df.head()

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
0,2020-02-29,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.01,2021-05-30 11:50:44.630549,1
1,2020-03-01,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30 11:50:44.630549,1
2,2020-03-02,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30 11:50:44.630549,1
3,2020-03-03,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.00,2021-05-30 11:50:44.630549,1
4,2020-03-04,5,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.03,0.01,2021-05-30 11:50:44.630549,1


In [59]:
ny_transform_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453 entries, 0 to 452
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Reported_Date                 453 non-null    datetime64[ns]
 1   New_Cases                     453 non-null    int64         
 2   mavg_7day_new_cases           453 non-null    int64         
 3   City                          453 non-null    object        
 4   Population                    453 non-null    int64         
 5   Data_Source                   453 non-null    object        
 6   new_cases_per_100K            453 non-null    float64       
 7   mavg_7day_per_100k_new_cases  453 non-null    float64       
 8   Extract_Date                  453 non-null    datetime64[ns]
 9   is_newyork                    453 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(2)
memory usage: 38.9+ KB


In [79]:
#6,7,8,9,10 add new static columns
ny_transform_df["City"] ="New York"
ny_transform_df["is_newyork"] =1
ny_transform_df["Population"] = 18804000
ny_transform_df["Data_Source"] ="https://github.com/nychealth/coronavirus-data/blob/master/trends/data-by-day.csv"
ny_transform_df["Extract_Date"] = dt.datetime.date(dt.datetime.utcnow())
ny_transform_df.head()

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
0,2020-02-29,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.01,2021-05-30,1
1,2020-03-01,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
2,2020-03-02,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
3,2020-03-03,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.00,2021-05-30,1
4,2020-03-04,5,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.03,0.01,2021-05-30,1


In [80]:
#before running window function to calculate rolling average check if reported date is in order
ny_transform_df

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
0,2020-02-29,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.01,2021-05-30,1
1,2020-03-01,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
2,2020-03-02,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
3,2020-03-03,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.00,2021-05-30,1
4,2020-03-04,5,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.03,0.01,2021-05-30,1
...,...,...,...,...,...,...,...,...,...,...
448,2021-05-22,220,342,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.17,1.82,2021-05-30,1
449,2021-05-23,170,328,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.90,1.74,2021-05-30,1
450,2021-05-24,327,311,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.74,1.65,2021-05-30,1
451,2021-05-25,277,294,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.47,1.56,2021-05-30,1


In [81]:
# just to be on safe side reorder the dataframe on the reproted date
ny_transform_df.sort_values(by='Reported_Date', inplace=True)
ny_transform_df

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
0,2020-02-29,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.01,2021-05-30,1
1,2020-03-01,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
2,2020-03-02,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
3,2020-03-03,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.00,2021-05-30,1
4,2020-03-04,5,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.03,0.01,2021-05-30,1
...,...,...,...,...,...,...,...,...,...,...
448,2021-05-22,220,342,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.17,1.82,2021-05-30,1
449,2021-05-23,170,328,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.90,1.74,2021-05-30,1
450,2021-05-24,327,311,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.74,1.65,2021-05-30,1
451,2021-05-25,277,294,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.47,1.56,2021-05-30,1


In [82]:
#11 Add caclulated column for per 100 K
ny_transform_df["new_cases_per_100K"]=round((ny_transform_df["New_Cases"]/ny_transform_df["Population"])*100000 ,2)
ny_transform_df 

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
0,2020-02-29,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.01,2021-05-30,1
1,2020-03-01,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
2,2020-03-02,0,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.00,0.00,2021-05-30,1
3,2020-03-03,1,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.01,0.00,2021-05-30,1
4,2020-03-04,5,0,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.03,0.01,2021-05-30,1
...,...,...,...,...,...,...,...,...,...,...
448,2021-05-22,220,342,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.17,1.82,2021-05-30,1
449,2021-05-23,170,328,New York,18804000,https://github.com/nychealth/coronavirus-data/...,0.90,1.74,2021-05-30,1
450,2021-05-24,327,311,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.74,1.65,2021-05-30,1
451,2021-05-25,277,294,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.47,1.56,2021-05-30,1


In [83]:
#12 Add rolling 7 days moving average for new_cases_per_100K
ny_transform_df["mavg_7day_per_100k_new_cases"] = round(ny_transform_df["new_cases_per_100K"].rolling(window=7,min_periods=1).mean(),2)
ny_transform_df.tail(15)

,Reported_Date,New_Cases,mavg_7day_new_cases,City,Population,Data_Source,new_cases_per_100K,mavg_7day_per_100k_new_cases,Extract_Date,is_newyork
438,2021-05-12,572,568,New York,18804000,https://github.com/nychealth/coronavirus-data/...,3.04,3.02,2021-05-30,1
439,2021-05-13,492,530,New York,18804000,https://github.com/nychealth/coronavirus-data/...,2.62,2.82,2021-05-30,1
440,2021-05-14,464,497,New York,18804000,https://github.com/nychealth/coronavirus-data/...,2.47,2.65,2021-05-30,1
441,2021-05-15,280,484,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.49,2.57,2021-05-30,1
442,2021-05-16,270,476,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.44,2.53,2021-05-30,1
443,2021-05-17,443,448,New York,18804000,https://github.com/nychealth/coronavirus-data/...,2.36,2.38,2021-05-30,1
444,2021-05-18,398,417,New York,18804000,https://github.com/nychealth/coronavirus-data/...,2.12,2.22,2021-05-30,1
445,2021-05-19,356,386,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.89,2.06,2021-05-30,1
446,2021-05-20,369,369,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.96,1.96,2021-05-30,1
447,2021-05-21,339,351,New York,18804000,https://github.com/nychealth/coronavirus-data/...,1.80,1.87,2021-05-30,1


In [85]:
# 13 Re order columns to create final data set
nyc_clean_df = ny_transform_df[["Extract_Date","Reported_Date","City","is_newyork","Population","Data_Source",
                          "New_Cases","mavg_7day_new_cases","new_cases_per_100K","mavg_7day_per_100k_new_cases"]]
nyc_clean_df

,Extract_Date,Reported_Date,City,is_newyork,Population,Data_Source,New_Cases,mavg_7day_new_cases,new_cases_per_100K,mavg_7day_per_100k_new_cases
0,2021-05-30,2020-02-29,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,1,0,0.01,0.01
1,2021-05-30,2020-03-01,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,0,0,0.00,0.00
2,2021-05-30,2020-03-02,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,0,0,0.00,0.00
3,2021-05-30,2020-03-03,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,1,0,0.01,0.00
4,2021-05-30,2020-03-04,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,5,0,0.03,0.01
...,...,...,...,...,...,...,...,...,...,...
448,2021-05-30,2021-05-22,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,220,342,1.17,1.82
449,2021-05-30,2021-05-23,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,170,328,0.90,1.74
450,2021-05-30,2021-05-24,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,327,311,1.74,1.65
451,2021-05-30,2021-05-25,New York,1,18804000,https://github.com/nychealth/coronavirus-data/...,277,294,1.47,1.56


In [86]:
#14 Export clean new york data as NYC_Covid_New_Cases_Final.csv
output_file = "Resources\\NYC_Covid_New_Cases_Final.csv"
nyc_clean_df.to_csv(output_file,index=False, header=True)

## Sao Paulo Data one time ETL

### Transformation requirements:

#### Sao Paulo data transformation requirements:
Note: ensure data is ordered by date before running rolling function.

    1.Extract data from source into a data frame, filter data to where codigo_ibge=3550308
    2.Validate the data types and data for datahora , casos_novos, pop
    3.Renmae column datahora as "Reported_Date"  
    4.Rename column casos_novos as "New_Cases"
    5.Rename column pop as "Population"
    6.Add new column City and populate all rows with value "Sao Paulo"
    7.Add new column is_newyork and populate all rows with value 0
    8.Add new column Data_Source and populate it with value https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv
    9.Add new column extract_date as todays date time to stamp the date the data is downloaded
    10.Add new calculated column new_cases_per_100K using formula (new_cases/population)*100000
    11.Add new calculated column mavg_7day_per_100k_new_cases using rolling function for pandas on new_cases_per_100K
    12.Add new calculated column mavg_7day_new_cases  using rolling function for pandas on new_cases column
    13.Change the order of columns to match "Extract_Date","Reported_Date","City","is_newyork","Population","Data_Source",
                          "New_Cases","mavg_7day_new_cases","new_cases_per_100K","mavg_7day_per_100k_new_cases"
    14.Export the final results as "SP_Covid_New_Cases_Final.csv" in Resources folder

In [89]:
#1. import directly from GitHub
sp_raw_df = pd.read_csv("https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv",parse_dates=['datahora'],sep=";")
sp_raw_df.head()

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,...,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
0,Adamantina,3500105,25,2,2020-02-25,0,0,"0,00000000000000e+00","0,000000000000000",0,...,Marília,5,33894,7398,41199,0,8.0,"-21,6820","-51,0737",9
1,Adolfo,3500204,25,2,2020-02-25,0,0,"0,00000000000000e+00","0,000000000000000",0,...,São José do Rio Preto,15,3447,761,21106,0,8.0,"-21,2325","-49,6451",9
2,Aguaí,3500303,25,2,2020-02-25,0,0,"0,00000000000000e+00","0,000000000000000",0,...,São João da Boa Vista,14,35608,5245,47455,0,8.0,"-22,0572","-46,9735",9
3,Águas da Prata,3500402,25,2,2020-02-25,0,0,"0,00000000000000e+00","0,000000000000000",0,...,São João da Boa Vista,14,7797,1729,14267,0,8.0,"-21,9319","-46,7176",9
4,Águas de Lindóia,3500501,25,2,2020-02-25,0,0,"0,00000000000000e+00","0,000000000000000",0,...,Campinas,3,18374,3275,6013,0,8.0,"-22,4733","-46,6314",9


In [95]:
#1 filter data for Sao Paulo where codigo_ibge=3550308
sp_raw_df = sp_raw_df[sp_raw_df['codigo_ibge'] == 3550308]  
sp_raw_df

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,...,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
562,São Paulo,3550308,25,2,2020-02-25,1,0,"8,42484114962012e-03","0,000000000000000",0,...,Grande São Paulo,10,11869660,1853286,152111,<50,7.0,"-23,5329","-46,6395",9
1207,São Paulo,3550308,26,2,2020-02-26,1,0,"8,42484114962012e-03","0,000000000000000",0,...,Grande São Paulo,10,11869660,1853286,152111,<50,7.0,"-23,5329","-46,6395",9
1852,São Paulo,3550308,27,2,2020-02-27,1,0,"8,42484114962012e-03","0,000000000000000",0,...,Grande São Paulo,10,11869660,1853286,152111,<50,7.0,"-23,5329","-46,6395",9
2497,São Paulo,3550308,28,2,2020-02-28,2,1,"1,68496822992402e-02","0,000000000000000",0,...,Grande São Paulo,10,11869660,1853286,152111,<50,7.0,"-23,5329","-46,6395",9
3142,São Paulo,3550308,29,2,2020-02-29,2,0,"1,68496822992402e-02","0,000000000000000",0,...,Grande São Paulo,10,11869660,1853286,152111,<50,7.0,"-23,5329","-46,6395",9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294037,São Paulo,3550308,25,5,2021-05-25,772503,2737,"6,50821506260499e+03","2359,142857142857338",30083,...,Grande São Paulo,10,11869660,1853286,152111,>5000,1.0,"-23,5329","-46,6395",21
294682,São Paulo,3550308,26,5,2021-05-26,775801,3298,"6,53600018871644e+03","2426,000000000000000",30192,...,Grande São Paulo,10,11869660,1853286,152111,>5000,1.0,"-23,5329","-46,6395",21
295327,São Paulo,3550308,27,5,2021-05-27,778119,2318,"6,55552897050126e+03","2319,000000000000000",30340,...,Grande São Paulo,10,11869660,1853286,152111,>5000,1.0,"-23,5329","-46,6395",21
295972,São Paulo,3550308,28,5,2021-05-28,778550,431,"6,55916007703675e+03","1965,714285714285779",30476,...,Grande São Paulo,10,11869660,1853286,152111,>5000,1.0,"-23,5329","-46,6395",21


In [96]:
#2. check columns data types
sp_raw_df.dtypes

nome_munic               object
codigo_ibge               int64
dia                       int64
mes                       int64
datahora         datetime64[ns]
casos                     int64
casos_novos               int64
casos_pc                 object
casos_mm7d               object
obitos                    int64
obitos_novos              int64
obitos_pc                object
obitos_mm7d              object
letalidade               object
nome_ra                  object
cod_ra                    int64
nome_drs                 object
cod_drs                   int64
pop                       int64
pop_60                    int64
area                      int64
map_leg                  object
map_leg_s               float64
latitude                 object
longitude                object
semana_epidem             int64
dtype: object

In [97]:
#validate
sp_raw_df[["datahora","codigo_ibge","casos_novos","pop"]].describe()

,codigo_ibge,casos_novos,pop
count,460.0,460.000000,460.0
mean,3550308.0,1697.117391,11869660.0
std,0.0,1415.756610,0.0
min,3550308.0,0.000000,11869660.0
25%,3550308.0,543.250000,11869660.0
50%,3550308.0,1397.500000,11869660.0
75%,3550308.0,2572.500000,11869660.0
max,3550308.0,8646.000000,11869660.0


In [98]:
# 3.Renmae column datahora as "Reported_Date"  
#4.Rename column casos_novos as "New_Cases"
#5.Rename column pop as "Population"
sp_transform_df = sp_raw_df[["datahora","casos_novos","pop"]]
sp_transform_df = sp_transform_df.rename(columns={"datahora":"Reported_Date",
                                               "casos_novos":"New_Cases",
                                               "pop":"Population"})
sp_transform_df.head()

,Reported_Date,New_Cases,Population
562,2020-02-25,0,11869660
1207,2020-02-26,0,11869660
1852,2020-02-27,0,11869660
2497,2020-02-28,1,11869660
3142,2020-02-29,0,11869660


In [99]:
#6,7,8,9  add new static columns
sp_transform_df["City"] ="Sao Paulo"
sp_transform_df["is_newyork"] =0
sp_transform_df["Data_Source"] ="https://raw.githubusercontent.com/seade-R/dados-covid-sp/master/data/dados_covid_sp.csv"
sp_transform_df["Extract_Date"] = dt.datetime.date(dt.datetime.utcnow())
sp_transform_df.head()

,Reported_Date,New_Cases,Population,City,is_newyork,Data_Source,Extract_Date
562,2020-02-25,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30
1207,2020-02-26,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30
1852,2020-02-27,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30
2497,2020-02-28,1,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30
3142,2020-02-29,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30


In [100]:
#10 Add caclulated column for per 100 K
sp_transform_df["new_cases_per_100K"]=round((sp_transform_df["New_Cases"]/sp_transform_df["Population"])*100000 ,2)
sp_transform_df 

,Reported_Date,New_Cases,Population,City,is_newyork,Data_Source,Extract_Date,new_cases_per_100K
562,2020-02-25,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1207,2020-02-26,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1852,2020-02-27,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
2497,2020-02-28,1,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.01
3142,2020-02-29,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
...,...,...,...,...,...,...,...,...
294037,2021-05-25,2737,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,23.06
294682,2021-05-26,3298,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,27.79
295327,2021-05-27,2318,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,19.53
295972,2021-05-28,431,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,3.63


In [101]:
#before running window function to calculate rolling average check if reported date is in order
sp_transform_df

,Reported_Date,New_Cases,Population,City,is_newyork,Data_Source,Extract_Date,new_cases_per_100K
562,2020-02-25,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1207,2020-02-26,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1852,2020-02-27,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
2497,2020-02-28,1,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.01
3142,2020-02-29,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
...,...,...,...,...,...,...,...,...
294037,2021-05-25,2737,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,23.06
294682,2021-05-26,3298,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,27.79
295327,2021-05-27,2318,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,19.53
295972,2021-05-28,431,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,3.63


In [102]:
# just to be on safe side reorder the dataframe on the reproted date
sp_transform_df.sort_values(by='Reported_Date', inplace=True)
sp_transform_df

,Reported_Date,New_Cases,Population,City,is_newyork,Data_Source,Extract_Date,new_cases_per_100K
562,2020-02-25,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1207,2020-02-26,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
1852,2020-02-27,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
2497,2020-02-28,1,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.01
3142,2020-02-29,0,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,0.00
...,...,...,...,...,...,...,...,...
294037,2021-05-25,2737,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,23.06
294682,2021-05-26,3298,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,27.79
295327,2021-05-27,2318,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,19.53
295972,2021-05-28,431,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,3.63


In [104]:
#11,12 Add rolling 7 days moving average for new_cases_per_100K and new cases
sp_transform_df["mavg_7day_per_100k_new_cases"] = round(sp_transform_df["new_cases_per_100K"].rolling(window=7,min_periods=1).mean(),2)
sp_transform_df["mavg_7day_new_cases"] = round(sp_transform_df["New_Cases"].rolling(window=7,min_periods=1).mean(),2)
sp_transform_df.tail(15)

,Reported_Date,New_Cases,Population,City,is_newyork,Data_Source,Extract_Date,new_cases_per_100K,mavg_7day_per_100k_new_cases,mavg_7day_new_cases
287587,2021-05-15,2732,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,23.02,19.33,2293.86
288232,2021-05-16,1544,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,13.01,19.64,2330.86
288877,2021-05-17,710,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,5.98,19.84,2355.43
289522,2021-05-18,2621,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,22.08,19.46,2309.71
290167,2021-05-19,2830,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,23.84,19.48,2312.86
290812,2021-05-20,3067,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,25.84,20.16,2393.14
291457,2021-05-21,2904,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,24.47,19.75,2344.00
292102,2021-05-22,3164,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,26.66,20.27,2405.71
292747,2021-05-23,914,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,7.70,19.51,2315.71
293392,2021-05-24,898,11869660,Sao Paulo,0,https://raw.githubusercontent.com/seade-R/dado...,2021-05-30,7.57,19.74,2342.57


In [105]:
# 13 Re order columns to create final data set
sp_clean_df = sp_transform_df[["Extract_Date","Reported_Date","City","is_newyork","Population","Data_Source",
                          "New_Cases","mavg_7day_new_cases","new_cases_per_100K","mavg_7day_per_100k_new_cases"]]
sp_clean_df

,Extract_Date,Reported_Date,City,is_newyork,Population,Data_Source,New_Cases,mavg_7day_new_cases,new_cases_per_100K,mavg_7day_per_100k_new_cases
562,2021-05-30,2020-02-25,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,0,0.00,0.00,0.00
1207,2021-05-30,2020-02-26,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,0,0.00,0.00,0.00
1852,2021-05-30,2020-02-27,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,0,0.00,0.00,0.00
2497,2021-05-30,2020-02-28,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,1,0.25,0.01,0.00
3142,2021-05-30,2020-02-29,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,0,0.20,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
294037,2021-05-30,2021-05-25,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,2737,2359.14,23.06,19.88
294682,2021-05-30,2021-05-26,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,3298,2426.00,27.79,20.44
295327,2021-05-30,2021-05-27,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,2318,2319.00,19.53,19.54
295972,2021-05-30,2021-05-28,Sao Paulo,0,11869660,https://raw.githubusercontent.com/seade-R/dado...,431,1965.71,3.63,16.56


In [106]:
#14 Export clean sao paulo data as SP_Covid_New_Cases_Final.csv
output_file = "Resources\SP_Covid_New_Cases_Final.csv"
sp_clean_df.to_csv(output_file,index=False, header=True)

In [ ]:
## Weather Data one time ETL

Notes:
    1.The weather data was a one time pull from openweathermap.org using their history bulk product. 
    2.The one time weather data from Jan 1st,2020 to May 16th,2021.
    3.The historical weather data is by hour.
     
### Transformation requirements:

#### Weather data transformation requirements:
Note: ensure data is ordered by date before running rolling function.

    1.Extract data from source into a data frame 
    2.Validate the data types and data for dt_iso , temp, humidity,temp_min,temp_max
    3.aggregate hourly data to daily using mean and keep only dt_iso,mean temp, min of temp_min,max of  temp_max, mean humidity drop all other weather columns 
    10.Add new calculated column new_cases_per_100K using formula (new_cases/population)*100000
    11.Add new calculated column mavg_7day_per_100k_new_cases using rolling function for pandas on new_cases_per_100K
    12.Add new calculated column mavg_7day_new_cases  using rolling function for pandas on new_cases column
    13.Change the order of columns to match "Extract_Date","Reported_Date","City","is_newyork","Population","Data_Source",
                          "New_Cases","mavg_7day_new_cases","new_cases_per_100K","mavg_7day_per_100k_new_cases"
    14.Export the final results as "SP_Covid_New_Cases_Final.csv" in Resources folder